In [4]:
import pandas as pd
import kagglehub
import nltk

nltk.download('stopwords')


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
def load_fiqa_dataset():
    splits = {
        'train': 'data/train-00000-of-00001-aeefa1eadf5be10b.parquet',
        'test': 'data/test-00000-of-00001-0fb9f3a47c7d0fce.parquet',
        'valid': 'data/valid-00000-of-00001-51867fe1ac59af78.parquet'
    }

    df_fiqa = pd.read_parquet("hf://datasets/TheFinAI/fiqa-sentiment-classification/" + splits["train"])

    df_fiqa = df_fiqa[['sentence', 'score']].rename(columns={'sentence': 'text', 'score': 'sentiment'})

    def convert_score_to_label(score):
        if score < 0:
            return -1
        elif score > 0:
            return 1
        else:
            return 0

    df_fiqa['sentiment'] = df_fiqa['sentiment'].apply(convert_score_to_label)
    print("FiQA:", df_fiqa.shape)

    # for label in [-1, 0, 1]:
    #     example = df_fiqa[df_fiqa["sentiment"] == label].iloc[2]["text"]
    #     print(f"Kaggle - First example for sentiment {label}:\n{example}\n")

    return df_fiqa


def load_kaggle_dataset():
    path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")
    kaggle_df = pd.read_csv(f"{path}/all-data.csv", encoding="ISO-8859-1", header=None)

    kaggle_df.columns = ["sentiment", "text"]
    sentiment_mapping = {"negative": -1, "neutral": 0, "positive": 1}
    kaggle_df["sentiment"] = kaggle_df["sentiment"].map(sentiment_mapping)

    print("Kaggle dataset:", kaggle_df.shape)

    # for label in [-1, 0, 1]:
    #     example = kaggle_df[kaggle_df["sentiment"] == label].iloc[0]["text"]
    #     print(f"Kaggle - First example for sentiment {label}:\n{example}\n")

    return kaggle_df


def load_all_datasets():
    # df_fiqa = load_fiqa_dataset()
    # df_kaggle = load_kaggle_dataset()
    # print("fiqa distirbution: ", df_fiqa['sentiment'].value_counts())
    # print("kaggle distirbution: ", df_kaggle['sentiment'].value_counts())


    # df_combined = pd.concat([df_fiqa, df_kaggle], ignore_index=True)
    # print("Sentiment class distribution:", df_combined['sentiment'].value_counts())
    # return df_combined

    all_financial_news_sentiment_local_dataset = pd.read_csv('/all_financial_news_sentiment_datasets.csv', encoding='utf-8', header=None)
    all_financial_news_sentiment_local_dataset.columns = ["text", "sentiment"]

    print(all_financial_news_sentiment_local_dataset.shape)
    #print(all_financial_news_sentiment_local_dataset.iloc[1]["sentiment"], all_financial_news_sentiment_local_dataset.iloc[1]["text"])
    return all_financial_news_sentiment_local_dataset

df = load_all_datasets()

#print(df.iloc[1]["text"], df.iloc[1].sentiment)

(14338, 2)


In [12]:
#preprocess data
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string
import re

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")

stop_words = set(stopwords.words('english'))
important_negations = {"not", "no", "nor", "never", "n't"}
stop_words = stop_words - important_negations

lemmatizer = WordNetLemmatizer()


def preprocess(text):
    if pd.isna(text) or text == "":
        return ""

    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

def handle_negations(text):
    # replace "not good" → "not_good"
    text = re.sub(r"\b(not|n't|never|no)\s+(\w+)", r"\1_\2", text)
    return text


df = df.dropna(subset=['text'])
df['text'] = df['text'].apply(preprocess)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [13]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import torch.optim as optim
from sklearn.model_selection import train_test_split


#embedder = SentenceTransformer('all-MiniLM-L6-v2')
embedder = SentenceTransformer('ProsusAI/finbert')
sampled_df = df.sample(n=len(df), random_state=42)
embeddings = embedder.encode(sampled_df["text"].tolist(), batch_size=32, show_progress_bar=True)

X = np.array(embeddings)
y = sampled_df['sentiment'].values.astype(np.float32)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Batches:   0%|          | 0/449 [00:00<?, ?it/s]

In [14]:
class SentimentRegressorClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

model = SentimentRegressorClassifier(input_dim=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [15]:
for epoch in range(15):
  model.train()
  train_loss = 0
  all_train_preds = []

  for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs.squeeze(), targets)
    loss.backward()
    optimizer.step()

    train_loss += loss.item() * inputs.size(0)
    all_train_preds.extend(outputs.view(-1).detach().cpu().numpy())

  avg_train_loss = train_loss / len(train_loader.dataset)
  model.eval()
  val_loss = 0
  all_val_preds = []

  with torch.no_grad():
    for inputs, targets in val_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs.squeeze(), targets)

      val_loss += loss.item() * inputs.size(0)
      all_val_preds.extend(outputs.view(-1).detach().cpu().numpy())

  avg_val_loss = val_loss / len(val_loader.dataset)
  scheduler.step(avg_val_loss)

In [16]:
def predict_sentiment_score(text):
  model.eval()
  vectorizer = embedder.encode([text])
  tensor = torch.tensor(vectorizer, dtype=torch.float32).to(device)

  with torch.no_grad():
    raw_score = model(tensor).item()
    print(raw_score)

  return round(raw_score, 3)

test_texts = [
    "US stock market couldn't crash anytime soon",
    "Trump increases chances of major stock prices downfall in US due to new tariffs",
    "Germany will spend more money on defend industry, weapons and army",
    "Trump administration could cause a major crash due to their new tariffs for all the countries."
]

print("\nTesting model predictions:")
for text in test_texts:
    score = predict_sentiment_score(text)
    print(f"Text: {text}...")
    print(f"Credibility score: {score}")
    print("---")


Testing model predictions:
0.7574200630187988
Text: US stock market couldn't crash anytime soon...
Credibility score: 0.757
---
-0.10934453457593918
Text: Trump increases chances of major stock prices downfall in US due to new tariffs...
Credibility score: -0.109
---
0.580921471118927
Text: Germany will spend more money on defend industry, weapons and army...
Credibility score: 0.581
---
0.25697430968284607
Text: Trump administration could cause a major crash due to their new tariffs for all the countries....
Credibility score: 0.257
---


In [17]:
import nltk

nltk.download('punkt_tab')
words2 = nltk.word_tokenize("Trump administration could cause a major crash due to their new tariffs for all the countries.")
res = []
for word in words2:
  if word not in stop_words:
    res.append(word)
print(res)

['Trump', 'administration', 'could', 'cause', 'major', 'crash', 'due', 'new', 'tariffs', 'countries', '.']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
